### 라이브러리 설치
- `lightgbm`

### 라이브러리 임포트

In [ ]:
from eli5.sklearn import PermutationImportance
import warnings;warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import platform

if platform.system() == 'Darwin':
    plt.rc('font', family='Apple SD Gothic Neo')


### 유틸리티 함수

In [ ]:
def vertical_text(text):
    return '\n'.join(text)

### 데이터 로드

In [ ]:
train = pd.read_csv('../../data/train.csv')
test = pd.read_csv('../../data/test.csv')

In [ ]:
train.shape, test.shape

### is_test 컬럼 마킹 + concat 통합 데이터 셋 생성

In [ ]:
train['is_test'] = 0
test['is_test'] = 1

concat = pd.concat([train, test])

In [ ]:
concat.head()

### correlation

In [ ]:
numeric_columns = []
for c in concat.columns.values:
    if pd.api.types.is_numeric_dtype(concat[c]):
        numeric_columns.append(c)
       
corr = concat[numeric_columns].corr()
mask = np.triu(np.ones_like(corr, dtype=bool))

plt.figure(figsize=(12, 12))
sns.heatmap(corr, mask=mask, fmt='0.2f', annot=True)

# 여기서 부터 다시 시작

In [ ]:
import warnings;warnings.filterwarnings('ignore')

import pandas as pd

In [ ]:
train_csv = pd.read_csv('../../data/train.csv')
test_csv = pd.read_csv('../../data/test.csv')

### info

In [ ]:
train_csv.info()

In [ ]:
test_csv.info()

### 결측치 분포

In [ ]:
concat_csv = pd.concat([
    train_csv.isnull().sum(),
    train_csv.isnull().mean(),
    test_csv.isnull().sum(),
    test_csv.isnull().mean(),
], axis=1)
concat_csv.columns = ['train_csv count', 'train_csv ratio', 'test_csv_count', 'test_csv ratio']
display('train_csv')
display(concat_csv.sort_values('train_csv ratio', ascending=False))
display('test_csv')
display(concat_csv.sort_values('test_csv ratio', ascending=False))


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# train과 test 데이터의 결측치 비율 계산
train_missing_ratio = train_csv.drop('target', axis=1).isnull().mean()
test_missing_ratio = test_csv.isnull().mean()

# 결측치 비율을 하나의 DataFrame으로 결합하여 시각화 준비
missing_data = pd.DataFrame({
    'feature': train_missing_ratio.index,
    'train_missing_ratio': train_missing_ratio.values,
    'test_missing_ratio': test_missing_ratio.values
})

In [ ]:
# 히스토그램 그리기
plt.figure(figsize=(20, 10))
sns.histplot(data=missing_data, x='feature', weights='train_missing_ratio',
             color='orange', label='Train Missing Ratio', multiple='stack', kde=False, alpha=0.5)
sns.histplot(data=missing_data, x='feature', weights='test_missing_ratio',
             color='blue', label='Test Missing Ratio', multiple='stack', kde=False, alpha=0.5)

# 그래프 설정
plt.xlabel('Features')
plt.ylabel('Missing Ratio')
plt.title('Missing Value Ratio per Feature in Train and Test Datasets')
plt.legend()

vertical_feature_names = [vertical_text(feature) for feature in missing_data['feature']]

plt.xticks(ticks=range(len(vertical_feature_names)), labels=vertical_feature_names, fontsize=16)
plt.tight_layout()

# 히스토그램 출력
plt.show()

#### 결론: 결측치 분포는 같다

### correlation 분석

In [ ]:
# 함수선언 
def draw_correlation_heatmap(dfs, threshold=0.8):
    fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(24, 12))
    
    for idx, df in enumerate(dfs):
        numeric_columns = []
        for c in df.columns.values:
            if pd.api.types.is_numeric_dtype(df[c]):
                numeric_columns.append(c)


        corr = df[numeric_columns].corr()
        mask = np.triu(np.ones_like(corr, dtype=bool))

        if threshold:
            mask |= (np.abs(corr) <= threshold)
            
        axs[idx].set_xticklabels(axs[idx].get_xticklabels(), rotation=270, fontsize=12)
        axs[idx].set_yticklabels(axs[idx].get_xticklabels(), fontsize=12)

        sns.heatmap(corr, mask=mask, fmt='0.2f', annot=True, cmap='coolwarm', ax=axs[idx])

In [ ]:
draw_correlation_heatmap([train_csv.drop('target', axis=1), test_csv], 0.75)

## 중간 가정
### correlation
#### 제거
- `k-` 시리즈 복잡하니 일단 제거한다
- `단지소개기존clob` 알아보기 귀찮으니 일단 제거한다

### 도메인에 따른
#### 제거
- `고용보험관리번호` 집 거래에 영향을 끼친다고 보기 힘들 것, 일단 제거한다
- `경비비관리형태` 집 거래에 영향을 끼친다고 보기 힘들 것, 일단 제거한다
- `세대전기계약방법` 집 거래에 영향을 끼친다고 보기 힘들 것, 일단 제거한다
- `청소비관리형태` 집 거래에 영향을 끼친다고 보기 힘들 것, 일단 제거한다
- `단지승인일` 집 거래에 영향을 끼친다고 보기 힘들 것, 일단 제거한다
- `단지신청일` 집 거래에 영향을 끼친다고 보기 힘들 것, 일단 제거한다
- `관리비 업로드`  집 거래에 영향을 끼친다고 보기 힘들 것, 일단 제거한다
- `좌표x` 메타 피쳐로 추가 파생이나 연결에 사용하고 주소가 있으니 일단 제거한다
- `좌표y` 메타 피쳐로 추가 파생이나 연결에 사용하고 주소가 있으니 일단 제거한다
- `기타/의무/임대/임의=1/2/3/4` **분석 필요** 
    - 결측치가 많은 것을 `기타` 로 설정하여 없앨 수 있으나 의미는 크지 않다
    - `기타` 를 제외한 값이 *discount* 역할을 할 수 있다. 동일 조건의 근처 아파트와 비교 필요
- `사용허가여부` **분석 필요**
  - 중요할 수 있으나 다른 `등기`, 혹은 실거래가 데이터인데 사용허가안난 건물도 거래하는지 들을 찾아볼 필요가 있다. 일단은 제거한다
- `계약일` 도 제거하는게 좋을 것 같다.

#### 변경
- `주차대수` 는 결측치가 많지만 주차가 세대수 대비 1이 안되는 경우 가격에 영향을 미칠 수 있다고 보고 결측치를 `1`로 가정 `1` 이하의 데이터 아파트와 구분한다

### 결측치에 따른
#### 제거
- `해제사유발생일` 결측치가 너무 많으므로 일단 제거하고, 시간이 날때에 해제사유 발생 유무 혹은 날짜 연산으로 변경한다

## 컬럼 제거 inplace


In [ ]:
features = [
    '시군구',
    '번지',
    '본번',
    '부번',
    '아파트명',
    '전용면적(㎡)',
    '계약년월',
    '계약일',
    '층',
    '건축년도',
    # '도로명',
    # '해제사유발생일',
    # '등기신청일자',
    # '거래유형',
    # '중개사소재지',
    # 'k-단지분류(아파트,주상복합등등)',
    # 'k-전화번호',
    # 'k-팩스번호',
    # '단지소개기존clob',
    # 'k-세대타입(분양형태)',
    # 'k-관리방식',
    # 'k-복도유형',
    # 'k-난방방식',
    # 'k-전체동수',
    # 'k-전체세대수',
    # 'k-건설사(시공사)',
    # 'k-시행사',
    # 'k-사용검사일-사용승인일',
    # 'k-연면적',
    # 'k-주거전용면적',
    # 'k-관리비부과면적',
    # 'k-전용면적별세대현황(60㎡이하)',
    # 'k-전용면적별세대현황(60㎡~85㎡이하)',
    # 'k-85㎡~135㎡이하',
    # 'k-135㎡초과',
    # 'k-홈페이지',
    # 'k-등록일자',
    # 'k-수정일자',
    # '고용보험관리번호',
    # '경비비관리형태',
    # '세대전기계약방법',
    # '청소비관리형태',
    '건축면적',
    '주차대수',
    # '기타/의무/임대/임의=1/2/3/4',
    # '단지승인일',
    # '사용허가여부',
    # '관리비 업로드',
    # '좌표X',
    # '좌표Y',
    # '단지신청일'
    # 'target'
]

In [ ]:
train_csv_v1 = train_csv[features]
test_csv_v1 = test_csv[features]


## 결측치 제거
- 결측치 제거 후 다시 확인한다

### 빈 문자열 등 의미 없는 문자를 제거한다
- 빈문자열 제거
  -`strip` 후의 `length` 내림차순

In [ ]:
# 숫자컬럼 유효성 검증
# numeric_columns = [c for c in s.columns if pd.api.types.is_numeric_dtype(s[c])]
# display(numeric_columns)
# [
# '본번',        0 -> NaN
# '부번',        0 -> NaN
# '전용면적(㎡)',  0 -> NaN
# '계약년월'      0 -> NaN
# '계약일',       0 -> NaN
# '층',          0 -> NaN(Lobby ?) -> 다른 데이터로 채워넣는게 나을듯
# '건축년도'       0 -> NaN 
# '건축면적'       0 -> NaN
# '주차대수'       NaN -> 1
# ]

# 본번 부번이 0.0으로 들어가있음 제거 필요

for index, _csv in enumerate((train_csv_v1, test_csv_v1)):
    # 본번 부번 번지가 모든 없는 경우 제거, 다른 데이터로 보간
    display(f'{index} 본번 부번')
    display(_csv[~_csv['본번'].isna() & _csv['번지'].isna()])
    display(_csv[(~_csv['본번'].isna() & _csv['부번'].isna()) & (_csv['본번'] .isna() & ~_csv['부번'].isna())])
    display(_csv[~_csv['본번'].isna() & _csv['번지'].isna()][['아파트명']].drop_duplicates())

    c = '전용면적(㎡)'
    ddf = _csv[_csv[c] .isna() & (_csv[c] <= 0)]
    ddf = ddf.sort_values(c)
    if len(ddf) > 0:
        display(f'{index} {c}')
        display(ddf.sort_values(c))
        display(ddf[['아파트명']].drop_duplicates())

    c = '계약년월'
    ddf = _csv[_csv[c] <= 0]
    if len(ddf) > 0:
        display(f'{index} {c}')
        display(ddf.sort_values(c))
        display(ddf[['아파트명']].drop_duplicates())
        
    c = '계약일'
    ddf = _csv[(_csv[c] <= 0) & (_csv[c] > 31)]
    if len(ddf) > 0:
        display(f'{index} {c}')
        display(ddf.sort_values(c))
        display(ddf[['아파트명']].drop_duplicates())
    
    c = '층'
    ddf = _csv[_csv['층'] <= 0]
    if len(ddf) > 0:
        display(f'{index} {c}')
        display(ddf.sort_values(c))
        display(ddf[['아파트명']].drop_duplicates())
    
    c = '건축년도'
    ddf = _csv[_csv[c] <= 0]
    if len(ddf) > 0:
        display(f'{index} {c}')
        display(ddf.sort_values(c))
        display(ddf[['아파트명']].drop_duplicates())
    
    # 0일수가 없다 모름 의미
    c = '건축면적'
    ddf = _csv[_csv[c] <= 0]
    if len(ddf) > 0:
        display(f'{index} {c}')
        display(ddf.sort_values(c))
        display(ddf[['아파트명']].drop_duplicates())

    # 0일수가 없다 모름 의미
    c = '주차대수'
    ddf = _csv[_csv[c] <= 0]
    if len(ddf) > 0:
        display(f'{index} {c}')
        display(ddf.sort_values(c))
        display(ddf[['아파트명']].drop_duplicates())


In [ ]:
for s in [train_csv_v1, test_csv_v1]:
    for c in s.columns:
        if not pd.api.types.is_numeric_dtype(s[c]):
            try:
                ddf = s[(s[c].str.strip().str.len().fillna(0) <= 2) & (s[c].str.contains(r'[-_]|^\s*$|없음', regex=True, na=True))]
                
                if len(ddf) > 0:
                    display(ddf[[c]].sort_values(by=c, key=lambda x: x.str.strip().str.len(), na_position='first').drop_duplicates())
                    display(ddf[c].nunique())
                    
            except Exception as e:
                print(c, 'error', s[c].dtype, e)



## 이상치 제거

In [ ]:
# 대표적인 연속형 변수인 “전용 면적” 변수 관련한 분포를 먼저 살펴보도록 하겠습니다.
fig = plt.figure(figsize=(7, 3))
sns.boxplot(data = train_csv_v1, x='전용면적(㎡)', color='lightgreen')
plt.title('전용면적 분포')
plt.xlabel('Area')
plt.show()
train_csv_v1.boxplot()

# 모델 테스트

In [ ]:
random_state = 42

In [ ]:
train_model_csv = train_csv_v1.copy()
test_model_csv = test_csv_v1.copy()

In [ ]:
from sklearn.preprocessing import LabelEncoder

c = '시군구'
le = LabelEncoder()
le.fit_transform(pd.concat([train_model_csv[c], test_model_csv[c]]))
train_model_csv[f'{c}_e'] = le.transform(train_model_csv[c])
test_model_csv[f'{c}_e'] = le.transform(test_model_csv[c])

c = '아파트명'
le = LabelEncoder()
le.fit_transform(pd.concat([ train_model_csv[c], test_model_csv[c]]))
train_model_csv[f'{c}_e'] = le.transform(train_model_csv[c])
test_model_csv[f'{c}_e'] = le.transform(test_model_csv[c])

In [ ]:
# 파생, 엔코딩 후 제거
train_model_csv = train_model_csv.drop(['시군구', '번지', '본번', '부번', '아파트명'], axis=1)
test_model_csv = test_model_csv.drop(['시군구', '번지', '본번', '부번', '아파트명'], axis=1)

# 결측치를 위해 임의로 제거
train_model_csv = train_model_csv.drop(['층', '계약일', '건축면적', '주차대수'], axis=1)
test_model_csv = test_model_csv.drop(['층', '계약일', '건축면적', '주차대수'], axis=1)

In [ ]:
display(train_model_csv.info())

train_model_csv

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    train_model_csv.drop('target', axis=1),
    train_model_csv['target'],
    test_size = 0.2,
    random_state=random_state,
)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(
    n_estimators=100,
    criterion='squared_error',
    random_state=random_state,
)

In [ ]:
%%time
model.fit(X_train, y_train)

In [ ]:
model.score(X_test, y_test)

In [ ]:
pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error

print(f'RMSE test: {np.sqrt(mean_squared_error(y_test, pred))}')

In [ ]:
pd.DataFrame({
    'column': train_model_csv.drop('target', axis=1).columns,
    'importance': model.feature_importances_
})


In [ ]:
pred_test = model.predict(test_model_csv)
preds_df = pd.DataFrame(pred_test.astype(int), columns=["target"])
preds_df.to_csv('output.csv', index=False)